# Construire des agents IA avec une mémoire persistante en utilisant Cognee

Ce notebook montre comment créer des agents IA intelligents avec des capacités de mémoire sophistiquées en utilisant [**cognee**](https://www.cognee.ai/) - une mémoire IA open source qui combine des graphes de connaissances, une recherche sémantique et une gestion de session pour créer des systèmes IA sensibles au contexte.

## 🎯 Objectifs d'apprentissage

À la fin de ce tutoriel, vous serez capable de :
- **Construire des graphes de connaissances basés sur des embeddings** : Transformer du texte non structuré en connaissances structurées et interrogeables
- **Mettre en œuvre une mémoire de session** : Créer des conversations multi-tours avec une rétention automatique du contexte
- **Conserver les conversations** : Stocker éventuellement des interactions importantes dans une mémoire à long terme pour une utilisation future
- **Interroger en langage naturel** : Accéder et exploiter le contexte historique dans de nouvelles conversations
- **Visualiser la mémoire** : Explorer les relations dans le graphe de connaissances de votre agent


## 🏗️ Ce que vous allez construire

Dans ce tutoriel, nous allons créer un **Assistant de Codage** avec une mémoire persistante qui :

### 1. **Construction d'une base de connaissances**
   - Intègre les informations sur le profil et l'expertise des développeurs
   - Traite les principes et les bonnes pratiques de programmation en Python
   - Stocke les conversations historiques entre les développeurs et les assistants IA

### 2. **Conversations conscientes du contexte**
   - Maintient le contexte à travers plusieurs questions dans une même session
   - Met automatiquement en cache chaque paire question/réponse pour une récupération efficace
   - Fournit des réponses cohérentes et contextuelles basées sur l'historique de la conversation

### 3. **Mémoire à long terme**
   - Conserve les conversations importantes dans une mémoire à long terme
   - Récupère des souvenirs pertinents de la base de connaissances et des sessions passées pour informer de nouvelles interactions
   - Construit une base de connaissances évolutive qui s'améliore avec le temps

### 4. **Récupération intelligente de la mémoire**
   - Utilise une recherche sémantique basée sur les graphes pour trouver des informations pertinentes dans toutes les connaissances stockées
   - Filtre les recherches par sous-groupes de données (informations sur les développeurs vs principes)
   - Combine plusieurs sources de données pour fournir des réponses complètes


## 📋 Prérequis et Configuration

### Configuration système

Avant de commencer, assurez-vous d'avoir :

1. **Environnement Python**
   - Python 3.9 ou supérieur
   - Environnement virtuel (recommandé)
   
2. **Cache Redis** (Requis pour la gestion des sessions)
   - Redis local : `docker run -d -p 6379:6379 redis`
   - Ou utilisez un service Redis géré
   
3. **Accès à l'API LLM**
   - Clé API OpenAI ou autres fournisseurs (voir [documentation](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Configuration de la base de données**
   - Aucune configuration requise par défaut. Cognee utilise des bases de données basées sur des fichiers (LanceDB et Kuzu)
   - Optionnellement, vous pouvez configurer Azure AI Search comme magasin de vecteurs (voir [documentation](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Configuration de l'environnement

Créez un fichier `.env` dans le répertoire de votre projet avec les variables suivantes :

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Comprendre l'Architecture Mémoire de Cognee

### Comment fonctionne Cognee

Cognee offre un système de mémoire sophistiqué qui va au-delà d'un simple stockage clé-valeur :

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Composants clés :

1. **Graph de Connaissances** : Stocke les entités, les relations et les connexions sémantiques
2. **Embeddings Vectoriels** : Permet la recherche sémantique à travers toutes les informations stockées
3. **Cache de Session** : Conserve le contexte des conversations au sein et entre les sessions
4. **NodeSets** : Organise les données en catégories logiques pour une récupération ciblée

### Types de mémoire dans ce tutoriel :

- **Mémoire Persistante** : Stockage à long terme dans le graph de connaissances
- **Mémoire de Session** : Contexte temporaire des conversations dans le cache Redis
- **Mémoire Sémantique** : Recherche de similarité basée sur des vecteurs à travers toutes les données


## 📦 Installer les paquets requis

Installez Cognee avec le support de Redis pour la gestion des sessions :


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Initialiser l'environnement et charger les bibliothèques

Assurez-vous que :
1. Redis est en cours d'exécution (par exemple, via Docker : `docker run -d -p 6379:6379 redis`)
2. Les variables d'environnement sont définies avant d'importer les modules de cache
3. Si nécessaire, redémarrez le noyau et exécutez les cellules dans l'ordre

La cellule suivante va :
1. Charger les variables d'environnement depuis `.env`
2. Configurer Cognee avec vos paramètres LLM
3. Activer la mise en cache pour la gestion des sessions
4. Valider que tous les composants sont correctement connectés


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Configurer les répertoires de stockage

Cognee utilise deux répertoires distincts pour ses opérations :
- **Racine des données** : Stocke les documents ingérés et les données traitées
- **Racine système** : Contient la base de données du graphe de connaissances et les métadonnées du système

Nous allons créer des répertoires isolés pour ce tutoriel comme suit :


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Réinitialiser l'état de la mémoire

Avant de commencer à construire notre système de mémoire, assurons-nous de partir sur une base saine.

> 💡 **Astuce** : Vous pouvez passer cette étape si vous souhaitez conserver les souvenirs existants de vos précédentes exécutions lorsque vous utiliserez ce notebook ultérieurement.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Partie 1 : Construire la base de connaissances

### Sources de données pour notre assistant développeur

Nous allons intégrer trois types de données pour créer une base de connaissances complète :

1. **Profil du développeur** : Expertise personnelle et parcours technique  
2. **Bonnes pratiques Python** : Le Zen de Python avec des directives pratiques  
3. **Conversations historiques** : Sessions de questions-réponses passées entre développeurs et assistants IA  

Ces données variées permettent à notre agent de :  
- Comprendre le contexte technique de l'utilisateur  
- Appliquer les meilleures pratiques dans ses recommandations  
- Apprendre des interactions réussies précédentes  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Transformer les données en graphe de connaissances

Nous allons maintenant transformer notre texte brut en une mémoire structurée. Ce processus :

1. **Ajoute des données aux NodeSets** : Organise les informations en catégories logiques
   - `developer_data` : Profil du développeur et conversations
   - `principles_data` : Bonnes pratiques et directives Python

2. **Exécute le pipeline Cognify** : Extrait les entités, les relations et crée des embeddings
   - Identifie les concepts clés
   - Crée des connexions sémantiques entre les informations liées
   - Génère des embeddings vectoriels

Cela peut prendre quelques instants pendant que le LLM traite le texte et construit la structure du graphe :


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualiser le graphe de connaissances

Explorons la structure de notre graphe de connaissances. La visualisation montre :
- **Nœuds** : Entités extraites du texte (concepts, technologies, personnes)
- **Arêtes** : Relations et connexions entre les entités
- **Clusters** : Concepts liés regroupés par similarité sémantique

Ouvrez le fichier HTML généré dans votre navigateur pour explorer le graphe de manière interactive :


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Enrichir la mémoire avec Memify

La fonction `memify()` analyse le graphe de connaissances et génère des règles intelligentes sur les données. Ce processus :
- Identifie des modèles et des bonnes pratiques
- Crée des directives exploitables basées sur le contenu
- Établit des relations entre différentes zones de connaissances

Ces règles aident l'agent à prendre des décisions plus éclairées lorsqu'il répond aux questions. Capturer une deuxième visualisation vous permet de comparer comment le graphe se densifie une fois enrichi.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Partie 2 : Récupération intelligente de la mémoire

### Démonstration 1 : Intégration des connaissances entre documents

Maintenant que notre graphe de connaissances est construit, testons comment Cognee combine des informations provenant de plusieurs sources pour répondre à des questions complexes.

La première requête démontre :
- **Compréhension sémantique** : Trouver des concepts pertinents même s'ils ne sont pas mentionnés explicitement
- **Référencement croisé** : Combiner le profil du développeur avec les principes de Python
- **Raisonnement contextuel** : Appliquer les meilleures pratiques à des implémentations spécifiques

### Démonstration 2 : Recherche filtrée avec NodeSets

La deuxième requête montre comment cibler des sous-ensembles spécifiques du graphe de connaissances :
- Utilise le paramètre `node_name` pour rechercher uniquement dans `principles_data`
- Fournit des réponses ciblées provenant d'un domaine de connaissances spécifique
- Utile lorsque vous avez besoin d'informations spécifiques à un domaine


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Partie 3 : Configuration de la gestion des sessions

### Activer la mémoire des conversations

La gestion des sessions est essentielle pour conserver le contexte au fil de plusieurs interactions. Ici, nous allons :

1. **Initialiser le contexte utilisateur** : Créer ou récupérer un profil utilisateur pour le suivi des sessions
2. **Configurer le moteur de cache** : Se connecter à Redis pour stocker l'historique des conversations
3. **Activer les variables de session** : Configurer des variables de contexte qui persistent entre les requêtes

> ⚠️ **Important** : Cela nécessite que Redis soit en cours d'exécution et que `CACHING=true` soit activé dans votre environnement


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Fonction d'aide : Voir l'historique de session

Cette fonction utilitaire nous permet d'inspecter l'historique des conversations stocké dans Redis. Elle est utile pour :
- Déboguer la gestion des sessions
- Vérifier que les conversations sont mises en cache
- Comprendre quel contexte est disponible pour l'agent


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Session 1 : Laboratoire de support asynchrone — Première question

Lancez la session `async-support-lab` en demandant des modèles asyncio adaptés à la télémétrie pour un énorme outil de scraping web. Le graphe connaît déjà asyncio, aiohttp et les pratiques de surveillance, donc la réponse devrait refléter les conversations précédentes tout en adaptant la réponse à cette nouvelle requête.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Inspecter la mémoire de la session 1 après le premier échange

Exécuter `show_history(session_1)` immédiatement après la question initiale confirme que Cognee a enregistré à la fois l'invite et la réponse dans Redis. Vous devriez voir une entrée avec les indications de concurrence.


In [ ]:
await show_history(session_1)

## Session 1 : Suivi sur les modèles de données

Ensuite, nous nous demandons : "Quand devrais-je choisir dataclasses plutôt que Pydantic ?" en utilisant le même identifiant de session. Cognee devrait rassembler les principes Python ainsi que les conversations précédentes sur FastAPI pour fournir des conseils nuancés—démontrant que le contexte se poursuit au sein d'une session nommée.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Confirmer que l'historique de la session 1 contient les deux échanges

Un autre appel à `show_history(session_1)` devrait révéler deux entrées de questions-réponses. Cela correspond à l'étape de "relecture de mémoire" du laboratoire Mem0 et prouve que des échanges supplémentaires prolongent le même transcript.


In [ ]:
await show_history(session_1)

## Session 2 : Fil de discussion sur la revue de conception — Nouvelle session

Pour montrer l'isolation entre les fils de discussion, nous lançons `design-review-session` et demandons des conseils de journalisation pour les revues d'incidents. Bien que la base de connaissances sous-jacente soit la même, le nouvel identifiant de session permet de garder les transcriptions séparées.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Révision de la session 2 : Histoire

`show_history(session_2)` doit uniquement afficher le couple de question/réponse de la revue de conception. Comparez cela avec la Session 1 pour mettre en évidence comment Cognee conserve des transcriptions indépendantes tout en réutilisant le graphe de connaissances partagé.


In [ ]:
await show_history(session_2)

## Résumé

Félicitations ! Vous venez d'offrir à votre assistant de codage une véritable couche de mémoire à long terme grâce à Cognee.

Dans ce tutoriel, vous avez pris du contenu brut de développeur (code, documents, discussions) et l'avez transformé en une mémoire graphique + vectorielle que votre agent peut rechercher, analyser et améliorer en continu.

Ce que vous avez appris :

1. **Du texte brut à la mémoire IA** : Comment Cognee ingère des données non structurées et les transforme en une mémoire intelligente et consultable grâce à une architecture combinée de vecteurs + graphe de connaissances.

2. **Enrichissement du graphe avec memify** : Comment aller au-delà de la création de graphe de base et utiliser memify pour ajouter des faits dérivés et des relations plus riches à votre graphe existant.

3. **Stratégies de recherche multiples** : Comment interroger la mémoire avec différents types de recherche (Q&R avec conscience du graphe, complétion de style RAG, insights, fragments bruts, recherche de code, etc.) en fonction des besoins de votre agent.

4. **Exploration visuelle** : Comment inspecter et déboguer ce que Cognee a construit en utilisant des visualisations de graphe et l'interface utilisateur de Cognee, afin de voir concrètement comment les connaissances sont structurées.

5. **Mémoire sensible aux sessions** : Comment combiner le contexte par session avec une mémoire sémantique persistante pour que les agents puissent se souvenir d'une session à l'autre sans divulguer d'informations entre les utilisateurs.


## Points Clés
1. Mémoire sous forme de Graphique de Connaissances soutenue par des Embeddings

   - **Compréhension structurée** : Cognee combine un magasin de vecteurs et un magasin de graphes pour que vos données soient à la fois recherchables par signification et connectées par des relations. Par défaut, Cognee utilise des bases de données basées sur des fichiers (LanceDB pour les vecteurs, Kuzu pour les bases de données graphiques).

   - **Récupération sensible aux relations** : Les réponses peuvent être basées non seulement sur du "texte similaire", mais aussi sur la manière dont les entités sont reliées.

   - **Mémoire vivante** : La couche mémoire évolue, grandit et reste interrogeable comme un graphique connecté unique.

2. Modes de Recherche et de Raisonnement
   - **Récupération hybride** : La recherche combine la similarité vectorielle, la structure du graphe et le raisonnement LLM, allant de la recherche brute de fragments à des réponses aux questions tenant compte du graphe.

   - **Adapter le mode à la tâche** : Utilisez les modes de type complétion lorsque vous souhaitez des réponses en langage naturel, et les modes fragment/résumé/graphe lorsque votre agent a besoin de contexte brut ou pour alimenter son propre raisonnement.

3. Agents Personnalisés et Sensibles aux Sessions
   - **Contexte de session + mémoire à long terme** : Cognee sépare le contexte à court terme des "fils de discussion" de la mémoire durable au niveau de l'utilisateur ou de l'organisation.

## Applications Réelles

1. **Agents IA Verticaux**

   Utilisez le modèle de ce notebook pour alimenter des copilotes intelligents dans leur domaine, reposant sur Cognee comme cœur de récupération et de raisonnement :

- **Copilotes pour développeurs** : Assistance pour la revue de code, l'analyse d'incidents et l'architecture, traversant le code, les API, les documents de conception et les tickets comme un seul graphique mémoire.

- **Copilotes orientés client** : Agents de support ou de succès qui puisent dans les documents produits, FAQ, notes CRM et tickets passés avec une récupération tenant compte des graphes et des réponses citées.

- **Copilotes experts internes** : Assistants en politique, juridique ou sécurité qui raisonnent sur des règles, directives et décisions historiques interconnectées au lieu de PDF isolés.

   Cognee se positionne explicitement comme une mémoire persistante et précise pour les agents IA, fournissant un graphique de connaissances vivant qui s'intègre derrière votre agent et remplace les combinaisons ad hoc de magasins de vecteurs et de code graphique personnalisé.

2. **Unifier les Silos de Données en une Mémoire Unique**

   La même approche vous aide également à construire une couche mémoire unifiée à partir de sources dispersées :

- **Des silos à un seul graphe** : Intégrez des données structurées (par ex., bases de données) et non structurées (par ex., documents, discussions) dans un seul graphe soutenu par des embeddings, plutôt que des index séparés pour chaque système.

- **Raisonnement inter-sources avec citations** : Effectuez un raisonnement en plusieurs étapes sur tout—"joignez" journaux, métriques et documents via le graphe—et retournez toujours des réponses fondées avec provenance.

- **Hubs de connaissances** : Pour des domaines comme la banque ou l'éducation, Cognee est déjà utilisé pour unifier des PDF, des systèmes internes et des données d'applications en un seul graphique de connaissances avec des vecteurs, permettant aux agents de répondre à des questions avec un contexte précis et cité.

## Prochaines Étapes

Vous avez mis en œuvre la boucle mémoire principale. Voici des extensions naturelles que vous pouvez essayer par vous-même (voir la [documentation Cognee](https://docs.cognee.ai/) pour plus de détails) :

1. **Expérimentez avec la conscience temporelle** : Activez la fonction de cognition temporelle pour extraire des événements et des horodatages à partir de texte.

2. **Introduisez un raisonnement basé sur une ontologie** : Définissez une ontologie OWL pour votre domaine. Utilisez le support d'ontologie de Cognee pour que les entités et relations extraites soient ancrées dans ce schéma, améliorant la qualité du graphe et les réponses spécifiques au domaine.

3. **Ajoutez une boucle de rétroaction** : Permettez à Cognee d'ajuster les poids des arêtes du graphe à partir des retours réels des utilisateurs, afin que la récupération s'améliore avec le temps au lieu de rester statique.

4. **Optimisez pour la personnalisation et le comportement en session** : Utilisez des identifiants d'utilisateur, des locataires et des ensembles de données pour offrir à chaque personne ou équipe sa propre vue sur le moteur de mémoire partagé.

5. **Étendez à des agents plus complexes** : Connectez Cognee à des frameworks d'agents pour construire des systèmes multi-agents partageant tous la même couche mémoire. *Le plugin Microsoft Agent Framework x Cognee arrive bientôt.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous ne sommes pas responsables des malentendus ou des interprétations erronées résultant de l'utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
